In [3]:
# Import base libraries
from numpy import *
from numpy.linalg import multi_dot
import pandas as pd
import yfinance as yf

# Import cufflinks
import cufflinks as cf
cf.set_config_file(offline=True, dimensions=((1000,600)))

# Import plotly express for EF plot
import plotly.express as px
px.defaults.template, px.defaults.width, px.defaults.height = "plotly_white", 1000, 600

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [69]:
# Asset list
symbols = ['JETFREIGHT.NS','RADHEDE.BO','ONTIC.BO','CRESSAN.BO','ROLLT.BO','SWORDEDGE.BO','VIVIDHA.BO','SANWARIA.BO','MFLINDIA.BO','PMCFIN.BO','SINTEXPLAST.BO','STAMPEDE.NS']

# Number of assets
numofasset = len(symbols)

# Number of portfolio for optimization
numofportfolio = 10000

In [70]:
# Fetch data from yahoo finance for last six years
bfassets = yf.download(symbols, start='2018-01-01', end='2023-05-29', progress=False)['Close']
bfassets.to_csv('bfassets15.csv')

# Load locally stored data
df= pd.read_csv('bfassets15.csv', index_col=0, parse_dates=True)

# Verify the output
df

,CRESSAN.BO,JETFREIGHT.NS,MFLINDIA.BO,ONTIC.BO,PMCFIN.BO,RADHEDE.BO,ROLLT.BO,SANWARIA.BO,SINTEXPLAST.BO,STAMPEDE.NS,SWORDEDGE.BO,VIVIDHA.BO
Date,,,,,,,,,,,,
2018-01-01,2.66,31.174999,0.21,10.50,0.45,21.549999,7.80,27.600000,83.150002,10.85,2.14,1.62
2018-01-02,2.66,29.625000,0.21,10.50,0.46,21.750000,7.71,28.950001,80.800003,11.35,2.14,1.57
2018-01-03,2.66,29.000000,0.21,10.50,0.55,21.700001,7.64,30.350000,81.199997,11.90,2.14,1.64
2018-01-04,2.66,30.000000,0.22,10.50,0.60,20.650000,7.26,31.850000,81.900002,12.45,2.24,1.72
2018-01-05,2.62,30.000000,0.22,10.50,0.66,19.799999,7.22,33.400002,87.099998,13.05,2.35,1.80
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-22,26.33,10.900000,0.66,0.56,1.92,2.990000,1.01,0.420000,1.870000,1.10,0.38,0.87
2023-05-23,25.84,10.950000,0.65,0.56,1.90,3.060000,1.04,NaN,1.870000,1.10,0.38,0.86
2023-05-24,25.00,11.000000,0.64,0.58,1.91,2.910000,1.04,NaN,1.870000,1.10,0.38,0.86


In [71]:
# Plot normalize price history
df['2018':].iplot(kind='line')

In [72]:
# Calculate returns 
returns = df.pct_change().fillna(0)
returns

# Plot annualized return and volatility
pd.DataFrame({
    'Annualized Return': round(returns.mean()*252*100,2),
    'Annualized Volatility': round(returns.std()*sqrt(252)*100,2)
}).iplot(kind='bar', shared_xaxes=True, subplots=True)

In [73]:
# Define Weights for Equal weighted portfolio
wts = array(numofasset * [1./numofasset])[:, newaxis]

# Derive portfolio returns, ret
ret = array(returns.mean()*252)[:, newaxis]

# Portfolio returns using @
wts.T @ ret

# Portfolio Variance & Volatility
cov = returns.cov()*252
var = multi_dot([wts.T, cov, wts])
sqrt(var)

array([[0.23059319]])

In [74]:
def portfolio_simulation(returns):

    # Initialize the lists
    rets = []; vols = []; wts = []

    # Simulate 5,000 portfolios
    for i in range(numofportfolio):

        # Generate random weights
        weights = random.random(numofasset)[:, newaxis]

        # Set weights such that sum of weights equals 1
        weights /= sum(weights)

        # Portfolio statistics
        rets.append(weights.T @ array(returns.mean()*252)[:, newaxis])
        vols.append(sqrt(multi_dot([weights.T, returns.cov()*252, weights])))
        wts.append(weights.flatten())

    # Create a dataframe for analysis
    portdf = 100*pd.DataFrame({
        'port_rets': array(rets).flatten(),
        'port_vols': array(vols).flatten(),
        'weights': list(array(wts))
    })
    
    portdf['sharpe_ratio'] = portdf['port_rets'] / portdf['port_vols']

    return round(portdf,2)

In [75]:
# Create a dataframe for analysis
temp = portfolio_simulation(returns)
temp

,port_rets,port_vols,weights,sharpe_ratio
0,-11.51,25.38,"[4.59407300829664, 13.777849719902665, 3.95790...",-0.45
1,1.74,25.55,"[5.237552927707425, 9.368149823514676, 7.47556...",0.07
2,-9.81,26.23,"[2.5608526471182613, 6.633299654986137, 8.9040...",-0.37
3,-0.40,25.08,"[14.610738183120786, 4.296393443981717, 5.4456...",-0.02
4,-8.64,27.58,"[5.605229368121987, 0.24572418353585704, 4.507...",-0.31
...,...,...,...,...
9995,1.58,27.10,"[0.07948066326793933, 17.819735197533046, 19.8...",0.06
9996,0.05,25.09,"[15.40092812992385, 17.082169123803837, 0.8504...",0.00
9997,7.12,25.66,"[11.846709803985874, 9.907905487019516, 16.338...",0.28
9998,-5.36,24.53,"[6.107562151501314, 5.958325977695565, 5.90440...",-0.22


In [76]:
# Get the max sharpe portfolio stats
temp.iloc[temp.sharpe_ratio.idxmax()]

port_rets                                                   23.06
port_vols                                                   25.44
weights         [18.516290542262322, 3.344549853488753, 19.017...
sharpe_ratio                                                 0.91
Name: 8268, dtype: object

In [77]:
# Verify the above result
temp.describe().T

,count,mean,std,min,25%,50%,75%,max
port_rets,10000.0,-1.170929,6.005129,-22.48,-5.1500,-1.16,2.89,23.06
port_vols,10000.0,25.022710,1.251549,22.05,24.1175,24.87,25.75,33.27
sharpe_ratio,10000.0,-0.045216,0.237463,-0.84,-0.2100,-0.05,0.12,0.91


In [78]:
# Max sharpe ratio portfolio weights
msrpwts = temp['weights'][temp['sharpe_ratio'].idxmax()]

# Allocation to achieve max sharpe ratio portfolio
dict(zip(symbols, msrpwts))

{'JETFREIGHT.NS': 18.516290542262322,
 'RADHEDE.BO': 3.344549853488753,
 'ONTIC.BO': 19.0173982452537,
 'CRESSAN.BO': 8.339206367978695,
 'ROLLT.BO': 15.90196728840394,
 'SWORDEDGE.BO': 12.666706462190547,
 'VIVIDHA.BO': 0.04362313091392792,
 'SANWARIA.BO': 1.2296002590604993,
 'MFLINDIA.BO': 1.656933442686301,
 'PMCFIN.BO': 4.262277031576711,
 'SINTEXPLAST.BO': 9.650752365401578,
 'STAMPEDE.NS': 5.370695010783018}

In [79]:
# Plot simulated portfolio
fig = px.scatter(
    temp, x='port_vols', y='port_rets', color='sharpe_ratio', 
    labels={'port_vols': 'Expected Volatility', 'port_rets': 'Expected Return','sharpe_ratio': 'Sharpe Ratio'},
    title="Monte Carlo Simulated Portfolio"
     ).update_traces(mode='markers', marker=dict(symbol='cross'))

# Plot max sharpe 
fig.add_scatter(
    mode='markers', 
    x=[temp.iloc[temp.sharpe_ratio.idxmax()]['port_vols']], 
    y=[temp.iloc[temp.sharpe_ratio.idxmax()]['port_rets']], 
    marker=dict(color='Red', size=20, symbol='star'),
    name = 'Max Sharpe'
).update(layout_showlegend=False)

# Show spikes
fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)
fig.show()

In [80]:
# Import optimization module from scipy
import scipy.optimize as sco

In [81]:
# Define portfolio stats function
def portfolio_stats(weights):
    
    weights = array(weights)[:,newaxis]
    port_rets = weights.T @ array(returns.mean() * 252)[:,newaxis]    
    port_vols = sqrt(multi_dot([weights.T, returns.cov() * 252, weights])) 
    
    return array([port_rets, port_vols, port_rets/port_vols]).flatten()

# Maximizing sharpe ratio
def min_sharpe_ratio(weights):
    return -portfolio_stats(weights)[2]

In [82]:
# Define initial weights
initial_wts = numofasset * [1./numofasset]
initial_wts

# Each asset boundary ranges from 0 to 1 bounds
bnds = tuple((0,1) for x in range(numofasset))

# Specify constraints
cons = ({'type': 'eq', 'fun': lambda x: sum(x)-1})

In [83]:
# Optimizing for maximum sharpe ratio
opt_sharpe = sco.minimize(min_sharpe_ratio, initial_wts, method='SLSQP', bounds=bnds, constraints=cons)
opt_sharpe

     fun: -1.4078914081265106
     jac: array([ 1.48236752e-04,  3.60966444e-01, -1.13919377e-04,  7.38525197e-01,
        2.07513571e-04, -1.62662566e-03,  8.69571775e-01,  2.25009871e+00,
        2.17635083e+00,  7.05841869e-01,  1.19974986e-01,  3.15227166e-01])
 message: 'Optimization terminated successfully'
    nfev: 119
     nit: 9
    njev: 9
  status: 0
 success: True
       x: array([5.01450465e-01, 0.00000000e+00, 2.40093501e-01, 7.65446734e-17,
       2.03615926e-01, 5.48401077e-02, 0.00000000e+00, 1.15163955e-15,
       1.43114687e-15, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00])

In [84]:
# Portfolio weights
list(zip(symbols, opt_sharpe['x']))

[('JETFREIGHT.NS', 0.501450465164387),
 ('RADHEDE.BO', 0.0),
 ('ONTIC.BO', 0.2400935013130791),
 ('CRESSAN.BO', 7.654467337747661e-17),
 ('ROLLT.BO', 0.20361592580327784),
 ('SWORDEDGE.BO', 0.05484010771925686),
 ('VIVIDHA.BO', 0.0),
 ('SANWARIA.BO', 1.1516395476141028e-15),
 ('MFLINDIA.BO', 1.4311468676808659e-15),
 ('PMCFIN.BO', 0.0),
 ('SINTEXPLAST.BO', 0.0),
 ('STAMPEDE.NS', 0.0)]

In [85]:
# Portfolio stats
stats = ['Returns', 'Volatility', 'Sharpe Ratio']
list(zip(stats, around(portfolio_stats(opt_sharpe['x']),4)))

[('Returns', 0.4553), ('Volatility', 0.3234), ('Sharpe Ratio', 1.4079)]

In [86]:
# Minimize the variance
def min_variance(weights):
    return portfolio_stats(weights)[1]**2

# Optimizing for minimum variance
opt_var = sco.minimize(min_variance, initial_wts, method='SLSQP', bounds=bnds, constraints=cons)
opt_var

     fun: 0.04738133276864755
     jac: array([0.09481918, 0.09527182, 0.09449748, 0.09456501, 0.09460361,
       0.09478612, 0.09477804, 0.09489333, 0.09470905, 0.0944499 ,
       0.09477621, 0.09463809])
 message: 'Optimization terminated successfully'
    nfev: 65
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([0.16201345, 0.12418657, 0.12629959, 0.07659617, 0.03309172,
       0.04804653, 0.08227409, 0.08063566, 0.04123813, 0.08029639,
       0.10146815, 0.04385354])

In [87]:
# Portfolio weights
list(zip(symbols, around(opt_var['x']*100,2)))

[('JETFREIGHT.NS', 16.2),
 ('RADHEDE.BO', 12.42),
 ('ONTIC.BO', 12.63),
 ('CRESSAN.BO', 7.66),
 ('ROLLT.BO', 3.31),
 ('SWORDEDGE.BO', 4.8),
 ('VIVIDHA.BO', 8.23),
 ('SANWARIA.BO', 8.06),
 ('MFLINDIA.BO', 4.12),
 ('PMCFIN.BO', 8.03),
 ('SINTEXPLAST.BO', 10.15),
 ('STAMPEDE.NS', 4.39)]

In [88]:
# Portfolio stats
stats = ['Returns', 'Volatility', 'Sharpe Ratio']
list(zip(stats, around(portfolio_stats(opt_var['x']),4)))

[('Returns', 0.0358), ('Volatility', 0.2177), ('Sharpe Ratio', 0.1646)]

In [89]:
def min_volatility(weights):
    return portfolio_stats(weights)[1]

In [90]:
# Efficient frontier params
targetrets = linspace(0.30,0.60,100)
tvols = []

for tr in targetrets:
    
    ef_cons = ({'type': 'eq', 'fun': lambda x: portfolio_stats(x)[0] - tr},
               {'type': 'eq', 'fun': lambda x: sum(x) - 1})
    
    opt_ef = sco.minimize(min_volatility, initial_wts, method='SLSQP', bounds=bnds, constraints=ef_cons)
    
    tvols.append(opt_ef['fun'])

targetvols = array(tvols)

# Dataframe for EF
efport = pd.DataFrame({
    'targetrets' : around(100*targetrets[14:],2),
    'targetvols': around(100*targetvols[14:],2),
    'targetsharpe': around(targetrets[14:]/targetvols[14:],2)
})

efport.head(5)

,targetrets,targetvols,targetsharpe
0,34.24,26.87,1.27
1,34.55,26.98,1.28
2,34.85,27.09,1.29
3,35.15,27.21,1.29
4,35.45,27.33,1.30


In [91]:
# Plot efficient frontier portfolio
fig = px.scatter(
    efport, x='targetvols', y='targetrets',  color='targetsharpe',
    labels={'targetrets': 'Expected Return', 'targetvols': 'Expected Volatility','targetsharpe': 'Sharpe Ratio'},
    title="Efficient Frontier Portfolio"
     ).update_traces(mode='markers', marker=dict(symbol='cross'))


# Plot maximum sharpe portfolio
fig.add_scatter(
    mode='markers',
    x=[100*portfolio_stats(opt_sharpe['x'])[1]], 
    y=[100*portfolio_stats(opt_sharpe['x'])[0]],
    marker=dict(color='red', size=20, symbol='star'),
    name = 'Max Sharpe'
).update(layout_showlegend=False)

# Plot minimum variance portfolio
fig.add_scatter(
    mode='markers',
    x=[100*portfolio_stats(opt_var['x'])[1]], 
    y=[100*portfolio_stats(opt_var['x'])[0]],
    marker=dict(color='green', size=20, symbol='star'),
    name = 'Min Variance'
).update(layout_showlegend=False)

# Show spikes
fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)
fig.show()